In [ ]:
import torch
print(torch.__version__)  # Should print the installed PyTorch version


In [ ]:
import torch
from torch import nn 
import matplotlib.pyplot as plt 
import numpy as np 

In [ ]:
print(torch.__version__)

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = "cpu"
print(f"Using device: {device}")

In [ ]:
m = 0.7
c = 0.3

X = torch.arange(0, 1, 0.02).unsqueeze(dim=1)
y = m*X + c
print("Shape of X = ",X.shape)
print("Shape of y = ",y.shape)

In [ ]:
split = int(len(X)*0.8)
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [6]:
def plot_data(X_train=X_train, y_train=y_train, X_test = X_test, y_test = y_test, predictions=None):

    plt.figure(figsize=(8,6))
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('X vs Y')
    plt.scatter(X_train, y_train, c='b', s=8, label='Training Data')
    plt.scatter(X_test, y_test, c='r', s=8, label='Testing Data')
    if predictions is not None:
        plt.scatter(X_test, predictions, c='g', s=8, label='Predicted Data')
 
    plt.legend()

In [ ]:
plot_data()

In [8]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(1, dtype=torch.float, requires_grad=True))
        self.bias = nn.Parameter(torch.randn(1, dtype=torch.float, requires_grad=True))
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.weights*x + self.bias

In [ ]:
torch.manual_seed(42)
model_0 = LinearRegressionModel()
list(model_0.parameters())

In [ ]:
with torch.inference_mode():
    y_pred = model_0(X_test)
plot_data(predictions=y_pred)

In [11]:
# Set up loss function and optimizer
loss_fn = nn.MSELoss()

optimizer = torch.optim.SGD(
    params = model_0.parameters(), lr = 0.1)

In [ ]:
model_0.to(device)
next(model_0.parameters()).device

In [ ]:
# Training Loop
epochs = 500
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)
trainLossList = []
testLossList = []
for epoch in range(epochs):
    ### Training 
    model_0.train()

    # 1. Forward Pass
    y_pred = model_0(X_train)

    # 2. Calculate Training Loss
    loss = loss_fn(y_train, y_pred)

    # 3. Zero grad optimizer
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Step optimizer
    optimizer.step()

    ### Testing
    with torch.inference_mode():
        test_preds = model_0(X_test)

        test_loss = loss_fn(y_test, test_preds)

    if epoch%10 == 0:
        print(f"Epoch: {epoch} | Training Loss: {loss:.4f} | Test Loss: {loss:.4f}")
    
    trainLossList.append(loss.cpu().detach().numpy())
    testLossList.append(test_loss.cpu().detach().numpy())

In [ ]:
with torch.inference_mode():
    y_pred = model_0(X_test)
plot_data(predictions=y_pred.cpu())

In [ ]:
# Plot Loss Curve
plt.figure(figsize=(10,8))
plt.plot(range(epochs), trainLossList, label="Training Loss")
plt.plot(range(epochs), testLossList, label="Test Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Epochs vs Loss")
plt.legend();